# Task 2: Understand body language by gesture recognition with convolutional neural network

## 1. Do literature search on Convolution Neural Network. Learn how to build a convolutional layer in PyTorch.

## 2. Referring to the guide in Task 1, build your own network for gesture classification using convolutional layers. Please see the references 4 in the manual to learn how to build convolutional layers in PyTorch.

## 3. Analyse and comment on the performance of the model. Make a comparison between the fully connected based and convolutional based models and comment on it.

In [1]:
import cv2
import numpy as np
import os
import itertools
import torch.utils.data as utils_data
import torch.nn as nn
import torch

In [2]:
path = './dataset/images'

label_path = './dataset/labels'
if not os.path.exists(label_path):
    os.makedirs(label_path)

files = os.listdir(path)
index = 0
for i, file in enumerate(files):
    if file != '.DS_Store':
        subclass_label_path = os.path.join(label_path, file + '.txt')
        with open(subclass_label_path, 'w') as f:
            f.write('#label\n')
        for _ in range(len(os.listdir(os.path.join(path, file)))):
            with open(subclass_label_path, 'a') as f:
                f.write('{:d}\n'.format(index))
        index = index + 1
    f.close()

In [3]:
Image = []
path_images = './dataset/images'
for mainDir, subDir, fileList in os.walk(path_images):
    for file in fileList:
        if file != '.DS_Store':
            currentPath = os.path.join(mainDir, file)
            Image.append(cv2.resize(cv2.imread(currentPath), (32, 32)))
Image = np.array(Image)
Image = np.transpose(Image, (0, 3, 1, 2))
dataset_size, C, H, W = Image.shape

Label = []
path_labels = './dataset/labels'
for file in os.listdir(path_labels):
    Label.append(np.loadtxt(os.path.join(path_labels, file)))
Label = np.array(list(itertools.chain.from_iterable(Label)))
num_classes = int(np.max(Label)) + 1

In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.module1 = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32*32,256),
            nn.Linear(256,64),
            nn.Linear(64,4)
        )

    def forward(self, x):
        output = self.module1(x)
        return output

In [5]:
model = CNNModel()
# if torch.cuda.is_available():
#     model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

Encapsulate data

In [6]:
dataset = utils_data.TensorDataset(torch.Tensor(Image), torch.LongTensor(Label))
split_ratio = 0.8
train_size = int(split_ratio * dataset_size)
test_size = dataset_size - train_size
train_set, test_set = utils_data.random_split(dataset, [train_size, test_size])

train_loader = utils_data.DataLoader(dataset=train_set, batch_size=8, shuffle=True)
test_loader = utils_data.DataLoader(dataset=test_set, batch_size=8, shuffle=True)

print('Data is ready!')

Data is ready!


In [7]:
best_accuracy = 0
for epoch in range(500):
    running_loss = 0.0
    train_acc = 0.0
    for step, (batch_image, batch_label) in enumerate(train_loader):
        model.train()
        # if torch.cuda.is_available():
        #     batch_image, batch_label = batch_image.cuda(), batch_label.cuda()
        batch_output = model(batch_image)
        batch_loss = loss_func(batch_output, batch_label)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

        running_loss += batch_loss.item()

        # train accuracy
        _, train_predicted = torch.max(batch_output.data, 1)
        train_acc += (train_predicted == batch_label).sum().item()

    train_acc /= train_size
    running_loss /= (step + 1)

    # ----------test----------
    model.eval()
    test_acc = 0.0
    for test_image, test_label in test_loader:
        test_output = model(test_image)
        _, predicted = torch.max(test_output.data, 1)
        test_acc += (predicted == test_label).sum().item()
    test_acc /= test_size

    print('epoch={:d}\ttrain loss={:.6f}\ttrain accuracy={:.3f}\ttest accuracy={:.3f}'.format(
        epoch, running_loss, train_acc, test_acc))

    if test_acc >= best_accuracy:
        torch.save(model.state_dict(), './trained_models/CNN_model.pkl')
        best_accuracy = test_acc

epoch=0	train loss=20.917231	train accuracy=0.403	test accuracy=0.500
epoch=1	train loss=1.337465	train accuracy=0.726	test accuracy=0.938
epoch=2	train loss=0.651810	train accuracy=0.806	test accuracy=1.000
epoch=3	train loss=0.537334	train accuracy=0.903	test accuracy=1.000
epoch=4	train loss=0.093872	train accuracy=0.968	test accuracy=1.000
epoch=5	train loss=0.058360	train accuracy=0.968	test accuracy=1.000
epoch=6	train loss=0.000007	train accuracy=1.000	test accuracy=1.000
epoch=7	train loss=0.000439	train accuracy=1.000	test accuracy=1.000
epoch=8	train loss=0.000255	train accuracy=1.000	test accuracy=1.000
epoch=9	train loss=0.000078	train accuracy=1.000	test accuracy=1.000
epoch=10	train loss=0.000021	train accuracy=1.000	test accuracy=1.000
epoch=11	train loss=0.000011	train accuracy=1.000	test accuracy=1.000
epoch=12	train loss=0.000009	train accuracy=1.000	test accuracy=1.000
epoch=13	train loss=0.000008	train accuracy=1.000	test accuracy=1.000
epoch=14	train loss=0.000007	

KeyboardInterrupt: 